# Garbage Collection (12/28 Pytorch study)

# Python 객체의 성질

In [1]:
import sys
import torch
print(sys.version)
print(torch.__version__)

3.6.12 |Anaconda, Inc.| (default, Sep  9 2020, 00:29:25) [MSC v.1916 64 bit (AMD64)]
1.7.1


In [2]:
a = 1
b = a
sys.getrefcount(a) # system 상에서 1을 많이 사용한다.

5061

In [3]:
a = 123123
b = a
print(sys.getrefcount(a))

4


In [4]:
del a
# print(sys.getrefcount(a))

In [5]:
print(b) # a를 삭제해도 b는 살아있다.
print(sys.getrefcount(b))

123123
2


In [6]:
a = "abcdabcd" # Python에서 String은 immutable
print(sys.getrefcount(a))
b = a
print(a is b) # 메모리 주소를 비교하는 "is" 연산
print(sys.getrefcount(a))

3
True
4


# 직접 객체를 만들어보자

VSCODE의 ***Show variables active in jupyter kernel*** 기능 사용!

In [2]:
# 객체 만들어보기
# 언제 생겨나고 죽는지 보기 위해서
class Test(object):
    def __init__(self):
        print("객체 생겼다.")
    
    def __del__(self):
        print("객체 죽었다.")


## object ?

> _객체 (Objects)_는 파이썬이 데이터(data)를 추상화한 것(abstraction)  

> 모든 객체는 아이덴티티(identity), 형(type), 값(value)을 갖습니다. 객체의 아이덴티티 는 한 번 만들어진 후에는 변경되지 않습니다. 메모리상에서의 객체의 주소로 생각해도 좋습니다.  
>〈is〉 연산자는 두 객체의 아이덴티티를 비교합니다; id() 함수는 아이덴티티를 정수로 표현한 값을 돌려줍니다.

[Python documents](https://docs.python.org/ko/3/reference/datamodel.html)

In [8]:
test1 = Test()
test2 = test1
del test1
print("화살표 하나만 지웠다고 메모리 사라지지 않음")
del test2
print("화살표를 모두 지우면 그제서야 삭제됨")

객체 생겼다.
화살표 하나만 지웠다고 메모리 사라지지 않음
객체 죽었다.
화살표를 모두 지우면 그제서야 삭제됨


In [9]:
test1 = Test()
test2 = Test()
del test1, test2


객체 생겼다.
객체 생겼다.
객체 죽었다.
객체 죽었다.


# Memory Leak (메모리 누수)

## Case 1 스스로 attribute 등으로 자기 자신을 가리키는 경우

In [11]:
test1 = Test()
test1.self_ref = test1 # Attribute로 자기 자신을 가리킴

객체 생겼다.


In [12]:
del test1 # 메모리에서 제거 되지는 않는데 더 이상 접근할 수 없음

## 이 때 사용하는 것이 Garbage collection

얘는 사실 강제로 호출할 수도 있지만 Python에서는 주기적으로 랜덤(?)하게 실행.

특징
- 느림
- 컴파일러가 해줌
- 알아서 랜덤하게 실행됨

In [13]:
import gc
gc.collect()

객체 죽었다.


2

## Case 2 서로 가리키는 경우

그래프 같은걸 그리다 보면 가끔 발생함.

In [17]:
class A(object):
    def __init__(self, obj):
        self.obj = obj # A.obj -> B
        obj.register(self) # B.obj -> A
    def __del__(self):
        print("A is DELETED")

class B(object):
    def register(self, obj):
        self.obj = obj
    def __del__(self):
        print("B is DELETED")

In [21]:
b = B()
a = A(b)

In [23]:
del a, b # 둘 다 삭제되지 않음

In [24]:
# 마찬가지로 garbage collection으로 해결
import gc
gc.collect()

B is DELETED
A is DELETED


32